<a href="https://colab.research.google.com/github/HemanthhVV/Apache-Beam/blob/main/Beam_NoteBook_I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install apache-beam[gcp]

In [ ]:
!pip install apache-beam

Understanding the concepts of Pipelines,Pcollections,Ptransforms

In [ ]:
#Basic Imports for the Further Processing
import argparse
import logging
import re

import apache_beam as beam
from apache_beam.io import ReadFromText,WriteToText
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import SetupOptions

In [ ]:
#Creating the class for the each transformation for the data
class Output(beam.PTransform):
    class _Transformation1(beam.DoFn):#Simply, DoFn is used for the creating Transformations
        def process(self,element):
            yield element," Transformation 1 Done 👌"
    class _Transformation2(beam.DoFn):
        def process(self,element):
            yield element," Transformation 2 Done "
    class _PrintingAfterTransformation(beam.DoFn):
        def process(self,element):
            print(list(','.join(tple) for tple in element) ," This is the final elemnt after the two transformations")
    def expand(self,input):#Arranging the transformations orders for our PTransform class
        input \#Pardo is used for applying the Transformataion which is created above
        | beam.ParDo(self._Transformation1()) \
        | beam.ParDo(self._Transformation2()) \
        | beam.ParDo(self._PrintingAfterTransformation())


In [ ]:
with beam.Pipeline() as p:
    (p | beam.Create(["Sample Test text"]) | Output())

['Sample Test text, Transformation 1 Done 👌', ' ,T,r,a,n,s,f,o,r,m,a,t,i,o,n, ,2, ,D,o,n,e, ']  This is the final elemnt after the two transformations


In [ ]:
with beam.Pipeline() as p:#Creating the inline Pcollections
    p | beam.Create([
        "Thus , HOw was the class?",
        "",
        "Enf of line",
        "That's it"
    ]) | beam.Filter(lambda line : line!="") | beam.Map(print)


Thus , HOw was the class?
Enf of line
That's it
